# Import and definitions

In [ ]:
# Jupyter Notebook 
from IPython.display import clear_output

# Additional utils
import numpy as np
import pprint

# c3 Imports
from c3po.signals.envelopes import *
from c3po.signals.component import Component as Comp
from c3po.signals.signal import IQ as IQ

from c3po.optimizer.optimizer import Optimizer as Optimizer

# Specify Initial Signal/Pulse and Bounds

### Initialize and set parameters + bounds of Components of the Signal/Pulse

In [ ]:
flattop_params = {
    'amp' : 15e6 * 2 * np.pi,
    'T_up' : 5e-9,
    'T_down' : 45e-9,
    'xy_angle' : 0,
    'freq_offset' : 0e6 * 2 * np.pi
}

flattop_bounds = {
    'amp': [1e3*2*np.pi, 1e9*2*np.pi],
    'T_up': [2e-9, 200e-9],
    'T_down': [2e-9, 200e-9],
    'xy_angle': [-np.pi, np.pi],
    'freq_offset': [-1e9*2*np.pi, 1e9*2*np.pi]
}

drag_params = {
    'amp' : 3e6 * 2 * np.pi,
    'T_up' : 25e-9,
    'T_down' : 30e-9,
    'xy_angle' : np.pi / 2.0,
    'freq_offset' : 0e6 * 2 * np.pi
}

drag_bounds = {
    'amp': [1e3*2*np.pi, 1e9*2*np.pi],
    'T_up': [2e-9, 200e-9],
    'T_down': [2e-9, 200e-9],
    'xy_angle': [-np.pi, np.pi],
    'freq_offset': [-1e9*2*np.pi, 1e9*2*np.pi]
}

carrier_parameters = {
    'freq' : 6e9 * 2 * np.pi
}


def my_flattop(t, params):
    t_up = params['T_up']
    t_down = params['T_down']
    return flattop(t, t_up, t_down)


# Component 1 of the signal
flat = Comp(
    desc = "pulse1",
    shape = my_flattop,
    params = flattop_params,
    bounds = flattop_bounds
)

# Each component has a Universally unique identifier (UUID) that identifies/represents the instance of the object
print("flat uuid: " + str(flat.get_uuid()))

# Component 2 of the signal
drag = Comp(
    desc = "pulse2",
    shape = my_flattop,
    params = drag_params,
    bounds = drag_bounds
)

# Each component has a Universally unique identifier (UUID) that identifies/represents the instance of the object
print("drag uuid: " + str(drag.get_uuid()))

# Component 3 of the signal; the carrier doesn't have a shape only parameters
carr = Comp(
    desc = "carrier",
    params = carrier_parameters
)

# Each component has a Universally unique identifier (UUID) that identifies/represents the instance of the object
print("carr uuid: " + str(carr.get_uuid()))

### Build Signal with Components 

In [ ]:
comps = []
comps.append(carr)
comps.append(flat)
comps.append(drag)

In [ ]:
sig = IQ()
sig.t_start = 0
sig.t_end = 150e-9
sig.res = [1e9, 1e12]

sig.calc_slice_num()
sig.create_ts()

sig.comps = comps

sig.save_params_to_history("initial")

In [ ]:
pprint.pprint(sig.get_parameters())

In [ ]:
#sig.plot_IQ_components()

In [ ]:
#sig.plot_fft_IQ_components()

In [ ]:
sig.plot_signal()

In [ ]:
#sig.plot_fft_signal()

# Calibration

In [ ]:
optim = Optimizer()

In [ ]:
opt_map = {
    'amp' : [(sig.get_uuid(), flat.get_uuid()), (sig.get_uuid(), drag.get_uuid())],
    'T_up' : [(sig.get_uuid(), flat.get_uuid()), (sig.get_uuid(), drag.get_uuid())],
    'T_down' : [(sig.get_uuid(), flat.get_uuid()), (sig.get_uuid(), drag.get_uuid())],
    'xy_angle': [(sig.get_uuid(), flat.get_uuid()), (sig.get_uuid(), drag.get_uuid())],
    'freq_offset': [(sig.get_uuid(), flat.get_uuid())]
}

In [ ]:
pprint.pprint(opt_map)

In [ ]:
opt_params = optim.get_corresponding_signal_parameters([sig], opt_map)

In [ ]:
pprint.pprint(opt_params)

In [ ]:
initial_spread = [ 5e6*2*np.pi, 5e6*2*np.pi, 5e-9, 5e-9, 5e-9, 5e-9, 0.1, 0.1, 20e6*2*np.pi]

opt_settings = {
    'CMA_stds': initial_spread,
    'ftarget' : 1e-4,
    'popsize' : 5
}

In [ ]:
import random

def evaluate_signals(samples_rescaled):
#     print(samples_rescaled)
#     i = 0
#     for sample in samples_rescaled:
#         sample.plot_signal()
#         print("sample: " + str(i))
#         i += 1
    return [random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)]

In [ ]:
optim = Optimizer()

In [ ]:
optim.optimize_signal(
    signals = [sig],
    opt_map = opt_map,
    opt = 'cmaes',
    settings = opt_settings,
    calib_name = 'test',
    eval_func = evaluate_signals
    )

In [ ]:
pprint.pprint(sig.get_history())